In [2]:
import torch
import math

import matplotlib.pyplot as plt
import openturns as ot
import numpy as np

import torch.nn as nn

import random

import torch.nn.functional as F

In [523]:
X = [14, 12, 13, 12]
outDim = [10,10]

test = SocialSig(X)

TypeError: __init__() takes 1 positional argument but 2 were given

In [640]:
class SocialSig():
    '''
    Class to create the social signature image
    '''
    def __init__(self):
        '''
        Basic steps for class:
            1. Randomly initalize 'weights' which I believe are actually the coords of the points
            2. Train kriggin model to interpolate points in between
            3. Predict what the points would be within a 224x224 matrix and output the resulting matrix
        '''
        super(SocialSig, self).__init__()
        self.outDim = [10,10]
        self.coords = torch.nn.Parameter(torch.tensor(random.choices(range(0,outDim[1]), k=16), dtype=torch.float32)) 

    def forward(self, input):
        self.X = input
        self.grid = self.__make_blank_coord_grid()
        tensorRet = self.IDW()
        return tensorRet
       
    
    def IDW(self):
        '''
        Train the IDW model to predict all of the points that are between known points
        '''
        self.coords = torch.clamp(self.coords, min=0, max=self.outDim[1])
        
        for cell in range(0, len(self.grid)):
            weightedVals = []
            for column in range(0, len(X)):
                xCoordLookup = column * 2
                yCoordLookup = xCoordLookup + 1
            

                measurementCellValue = self.X[column]

                estCellX = self.grid[cell][0]
                estCellY = self.grid[cell][1]

                measureCellX = self.coords[xCoordLookup]
                measureCellY = self.coords[yCoordLookup]

                A2 = abs(estCellX - measureCellX)**2
                B2 = abs(estCellY - measureCellY)**2
                C2 = math.sqrt(A2+B2) 
                if(C2 == 0):
                    C2 = 1
                
                weightedVals.append(measurementCellValue[0] * (1/(C2**2)))
            self.grid[cell] = sum(weightedVals)
        numpyGrid = torch.from_numpy(np.reshape(np.array(self.grid), (1,1,self.outDim[0],self.outDim[1])))
        tensorGrid = torch.tensor(numpyGrid, dtype=torch.float)
        print(self.coords)

        return tensorGrid 
        
    def __make_blank_coord_grid(self):
        '''
        Make a blank coordinate grid to fill in with real data later
        '''
        return [[x,y] for x in range(0, outDim[0]) for y in range(0,outDim[1])]

In [641]:
# THIS IS WHAT YOU WILL USE TO CREATE YOUR INPUT LAYER--THIS SHOULD BE A LIST WITH 81 ELEMENTS (THE ROW IN THE DF)
def make_example_data(seed, size):
    '''
    Create the training dataset by randomly assigning coordiantes to the known data (i.e the data within a row)
    '''
    points = [[i] for i in list(np.random.randint(low = 0, high = 255, size = size).flatten())]
    return points


make_example_data(14, size=4)

[[223], [146], [68], [196]]

In [660]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.SocialSig = SocialSig()
        self.conv1 = nn.Conv2d(1, 10, 10, 1)
        self.linear1 = torch.nn.Linear(10, 1)
        self.linear2 = torch.nn.Linear(2, 1)

    def forward(self, input):
        print(input)
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        sig = self.SocialSig.forward(input)
        print(sig)
        sig = self.conv1(sig)
        print(sig.size()) #1,10,1,1
        sig = torch.flatten(sig, 1)
        print(sig.size())#1,10
        sig = self.linear1(sig)

        return sig

In [661]:
# Construct our model by instantiating the class defined above
x = make_example_data(14, size=4)
model = TwoLayerNet()

In [664]:
x = make_example_data(14, size=4)
y = torch.tensor([1], dtype=torch.float32)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(5):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)
    
    print("PRED: ", y_pred)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print("\n")

[[195], [226], [87], [183]]
tensor([2., 8., 9., 5., 6., 9., 0., 1., 8., 0., 4., 3., 0., 0., 4., 6.],
       grad_fn=<ClampBackward>)
tensor([[[[188.7433, 189.8791, 191.4096,  56.3413,  34.2657,  30.9007,  36.3844,
            48.9172,  57.3472,  46.6059],
          [ 97.8601, 190.7025, 101.0418,  48.8498,  34.9875,  35.9179,  52.0742,
           108.7695, 205.1021, 106.6204],
          [ 43.5978,  54.2940,  47.2517,  36.6122,  32.9064,  38.1477,  63.0603,
           208.1891, 207.4670, 206.6056],
          [ 25.9716,  29.7713,  30.0925,  29.1603,  30.3042,  36.5778,  55.3238,
           113.9090, 211.8774, 114.0197],
          [ 19.1759,  21.9084,  23.9283,  25.8422,  28.7623,  34.1087,  44.2230,
            61.1873,  75.6124,  68.5497],
          [ 16.2829,  19.0830,  22.1518,  25.6970,  29.8226,  34.5394,  40.6541,
            51.2000,  76.6796, 115.6187],
          [ 15.1046,  18.4827,  23.0270,  29.1324,  36.2404,  41.8678,  45.0167,
            53.1469, 113.8960, 109.3406],
      

In [665]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
x = SocialSig()

In [666]:
'''
Make a layer that has a x trainabale params as the coordinates
    -- THIS LAYER NEEDS TO HAVE A FORWARD PASS
    (https://discuss.pytorch.org/t/pytorch-equivalent-of-keras/29412)
'''
 

'\nMake a layer that has a x trainabale params as the coordinates\n    -- THIS LAYER NEEDS TO HAVE A FORWARD PASS\n    (https://discuss.pytorch.org/t/pytorch-equivalent-of-keras/29412)\n'